<a href="https://colab.research.google.com/github/StevenLevine-NOAA/NBM-Verif/blob/notebooks/Probs_and_Obs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Initialize Notebook Part 1
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#@title Initialize Notebook Part 2
#!mamba install -q -c conda-forge cartopy contextily pyproj pyepsg pygrib netCDF4
!pip install cartopy contextily pyproj pyepsg pygrib netCDF4
import numpy as np
from scipy.interpolate import CubicSpline as cs, UnivariateSpline as us
import pandas as pd
from urllib.request import urlretrieve, urlopen
import requests
from datetime import datetime, timedelta
import json
from netCDF4 import Dataset
import pygrib
import pyproj
from pyproj import Proj, transform
import os, re, traceback

import matplotlib
from matplotlib.colors import LinearSegmentedColormap
#from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.axes as maxes
import matplotlib.patheffects as PathEffects
from matplotlib.path import Path
from matplotlib.textpath import TextToPath
import matplotlib.gridspec as gridspec
from matplotlib.font_manager import FontProperties
matplotlib.rcParams['font.sans-serif'] = 'Liberation Sans'
matplotlib.rcParams['font.family'] = "sans-serif"
from matplotlib.cm import get_cmap
import seaborn as sns

from cartopy import crs as ccrs, feature as cfeature
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
import contextily as cx
import itertools

import zipfile

import warnings
warnings.filterwarnings("ignore")

In [16]:
#@title Select Options and Set Defaults

element = "maxt" #@param ["maxt", "mint","qpf","maxwind","snow"]
valid_date = "2024-01-18" #@param {type:"date"}
qpf_valid_time = 12 #@param {type:"slider", min:0, max:18, step:6}
use_stageiv = True #@param {type:"boolean"}
use_nohrsc = True #@param {type:"boolean"}
nbm_init_date = "2024-01-13" #@param {type:"date"}
nbm_init_hour = 0 #@param {type:"slider", min:0, max:18, step:6}
region_selection = "CR" #@param ["WR", "SR", "CR", "ER", "AR", "CONUS", "CWA"]
cwa_id = "" #@param {type:"string"}
network_selection = "NWS" #@param ["NWS", "RAWS", "NWS+RAWS", "NWS+RAWS+HADS", "ALL", "CUSTOM", "LIST"]
cwa_outline = True #@param {type:"boolean"}
county_outline = False #@param {type:"boolean"}
export_csv = True #@param {type:"boolean"}
#@markdown Light or dark theme plots?
plot_style = "dark" #@param ["light", "dark"]

if region_selection == "CONUS":
  region_list = ["WR", "CR", "SR", "ER"]
elif region_selection == "CWA":
  region_list = [cwa_id]
#elif region_selection == "AR":
  #region_list=["AJK","ARH","AFC"]
else:
  region_list = [region_selection]

def cwa_list(input_region):
  region_dict ={"WR":"BYZ,BOI,LKN,EKA,FGZ,GGW,TFX,VEF,LOX,MFR,MTR,MSO,PDT,PSR,PIH,PQR,REV,STO,SLC,SGX,HNX,SEW,OTX,TWC",
              "CR":"ABR,BIS,CYS,LOT,DVN,BOU,DMX,DTX,DDC,DLH,FGF,GLD,GJT,GRR,GRB,GID,IND,JKL,EAX,ARX,ILX,LMK,MQT,MKX,MPX,LBF,APX,IWX,OAX,PAH,PUB,UNR,RIW,FSD,SGF,LSX,TOP,ICT",
              "ER":"ALY,LWX,BGM,BOX,BUF,BTV,CAR,CTP,RLX,CHS,ILN,CLE,CAE,GSP,MHX,OKX,PHI,PBZ,GYX,RAH,RNK,AKQ,ILM",
              "SR":"ABQ,AMA,FFC,EWX,BMX,BRO,CRP,EPZ,FWD,HGX,HUN,JAN,JAX,KEY,MRX,LCH,LZK,LUB,MLB,MEG,MFL,MOB,MAF,OHX,LIX,OUN,SJT,SHV,TAE,TBW,TSA",
              "AR":"AJK,ARH,AFC"}
  if (input_region in ["WR", "CR", "SR", "ER","AR"]):
    cwas_list = region_dict[input_region]
  else:
    cwas_list = input_region
  return cwas_list

nbm_init = datetime.strptime(nbm_init_date,'%Y-%m-%d') + timedelta(hours=int(nbm_init_hour))

if element == "maxt":
    nbm_core_valid_hour="00"
    nbm_qmd_valid_hour="06"
    valid_date_start = datetime.strptime(valid_date,'%Y-%m-%d')
    valid_date_end = datetime.strptime(valid_date,'%Y-%m-%d') + timedelta(days=1)
    obs_start_hour = "1200"
    obs_end_hour = "0600"
    ob_stat = "maximum"
    valid_end_datetime = valid_date_end + timedelta(hours=(int(obs_end_hour)/100))
    nbm_core_valid_end_datetime = valid_date_end + timedelta(hours=int(nbm_core_valid_hour))
    nbm_qmd_valid_end_datetime = valid_date_end + timedelta(hours=int(nbm_qmd_valid_hour))
    core_init = nbm_init + timedelta(hours = 7)
    nbm_core_fhdelta = nbm_core_valid_end_datetime - core_init

elif element == "mint":
    nbm_core_valid_hour="12"
    nbm_qmd_valid_hour="18"
    valid_date_start = datetime.strptime(valid_date,'%Y-%m-%d')
    valid_date_end = datetime.strptime(valid_date,'%Y-%m-%d')
    obs_start_hour = "0000"
    obs_end_hour = "1800"
    ob_stat = "minimum"
    valid_end_datetime = valid_date_end + timedelta(hours=(int(obs_end_hour)/100))
    nbm_core_valid_end_datetime = valid_date_end + timedelta(hours=int(nbm_core_valid_hour))
    nbm_qmd_valid_end_datetime = valid_date_end + timedelta(hours=int(nbm_qmd_valid_hour))
    core_init = nbm_init + timedelta(hours = 7)
    nbm_core_fhdelta = nbm_core_valid_end_datetime - core_init

elif element == "qpf":
    nbm_core_valid_hour = (str(qpf_valid_time)).zfill(2)
    nbm_valid_hour = (str(qpf_valid_time)).zfill(2)
    nbm_qmd_valid_hour=(str(qpf_valid_time)).zfill(2)
    valid_date = datetime.strptime(valid_date,'%Y-%m-%d') + timedelta(hours=int(qpf_valid_time))
    valid_date_start = valid_date - timedelta(hours=24)
    valid_date_end = valid_date
    obs_start_hour = (str(qpf_valid_time)).zfill(2)+"00"
    obs_end_hour = (str(qpf_valid_time)).zfill(2)+"00"
    ob_stat = "total"
    valid_end_datetime = valid_date_end
    core_init = nbm_init
    nbm_core_valid_end_datetime = valid_date_end
    nbm_qmd_valid_end_datetime = valid_date_end
    nbm_core_fhdelta = nbm_core_valid_end_datetime - nbm_init

elif element == "maxwind":
    #nbm_core_valid_hour="06"
    #nbm_valid_hour="06"
    nbm_qmd_valid_hour="06"
    obs_start_hour="0600"
    obs_end_hour="0600"
    ob_stat="maximum"
    valid_date_start = datetime.strptime(valid_date,'%Y-%m-%d')
    valid_date_end = datetime.strptime(valid_date,'%Y-%m-%d') + timedelta(days=1)
    valid_end_datetime=valid_date_end + timedelta(hours=(int(obs_end_hour)/100))
    core_init = nbm_init
    nbm_core_valid_end_datetime = valid_date_end
    nbm_qmd_valid_end_datetime = valid_date_end + timedelta(hours=int(nbm_qmd_valid_hour))
    nbm_core_fhdelta = valid_end_datetime - nbm_init
    #valid_date=date.strptime(valid_date,'') + timedelta(hours=)

elif element == "snow":# or element == "ice":
    nbm_core_valid_hour=(str(qpf_valid_time)).zfill(2)
    nbm_qmd_valid_hour=(str(qpf_valid_time)).zfill(2)
    obs_start_hour=(str(qpf_valid_time)).zfill(2)+"00"
    obs_end_hour = (str(qpf_valid_time)).zfill(2)+"00"
    valid_date = datetime.strptime(valid_date,'%Y-%m-%d') #+ timedelta(hours=int(qpf_valid_time))
    valid_date_start = valid_date - timedelta(hours=24)
    valid_date_end = valid_date
    valid_end_datetime = valid_date_end + timedelta(hours=(int(obs_end_hour)/100))
    ob_stat = "total"
    core_init = nbm_init + timedelta(hours = 1)
    nbm_qmd_valid_end_datetime = valid_date_end + timedelta(hours=int(nbm_qmd_valid_hour))
    nbm_core_valid_end_datetime = nbm_qmd_valid_end_datetime
    nbm_core_fhdelta = nbm_core_valid_end_datetime - core_init

current_datetime=datetime.now()

nbm_qmd_fhdelta = nbm_qmd_valid_end_datetime - nbm_init
nbm_qmd_forecasthour = nbm_qmd_fhdelta.total_seconds() / 3600.
if element == "qpf" or element == "maxwind" or element == "maxgust" or element == "snow" or element == "ice24":
  nbm_qmd_forecasthour_start = nbm_qmd_forecasthour - 24
else:
  nbm_qmd_forecasthour_start = nbm_qmd_forecasthour - 18

# Setup a dictionary for translating a form selection into a something we can pass to mesowest API
network_dict = {"NWS+RAWS+HADS":"&network=1,2,106","NWS+RAWS":"&network=1,2", "NWS":"&network=1", "RAWS": "&network=2", "ALL":""}
network_string = network_dict[network_selection]

if element == "qpf":
  cmap = get_cmap('BrBG')
  cmap.set_under(color='black')
  cmap.set_over(color='yellow')
elif element == "snow":
  cmap = get_cmap('cool_r')
  cmap.set_under(color='black')
  cmap.set_over(color='yellow')
else:
  #cmap = 'Spectral'
  cmap = get_cmap('bwr')
  cmap.set_under(color='yellow')
  cmap.set_over(color='black')
if use_stageiv and element=="qpf":
  points_str = f'Stage IV @ {network_selection}'
else:
  points_str = network_selection

if plot_style=="light":
  background_color = '#f7f7f7'
  text_color = '#121212'
  map_land_color = '#FAFAF8'
  map_water_color = '#D4DBDD'
  map_border_color = 'grey'
elif plot_style=="dark":
  background_color = '#272727'
  text_color = 'white'
  map_land_color = '#414143'
  map_water_color = '#272727'
  #map_border_color = '#3B3B3D'
  map_border_color = 'white'

########################################################################################################################
# Reusable functions section                                                                                           #
########################################################################################################################

def project3(lon, lat, prj):
  lon = float(lon)
  lat = float(lat)

  outproj = prj
  inproj = Proj(init='epsg:4326')
  nbm_coords = transform(inproj, outproj, lon, lat)
  coordX = nbm_coords[0]
  coordY = nbm_coords[1]
  #print(f'Lat: {lat}, Y: {coordY} | Lon: {lon}, X: {coordX}')
  return(coordX, coordY)


def ll_to_index(datalons, datalats, loclon, loclat):
  abslat = np.abs(datalats-loclat)
  abslon = np.abs(datalons-loclon)
  c = np.maximum(abslon, abslat)
  latlon_idx_flat = np.argmin(c)
  latlon_idx = np.unravel_index(latlon_idx_flat, datalons.shape)
  return(latlon_idx)


def project_hrap(lon, lat, s4x, s4y):
  lon = float(lon)
  lat = float(lat)

  globe = ccrs.Globe(semimajor_axis=6371200)
  hrap_ccrs = proj = ccrs.Stereographic(central_latitude=90.0,
                          central_longitude=255.0,
                          true_scale_latitude=60.0, globe=globe)
  latlon_ccrs = ccrs.PlateCarree()
  hrap_coords = hrap_ccrs.transform_point(lon,lat,src_crs=latlon_ccrs)
  hrap_idx = ll_to_index(s4x, s4y, hrap_coords[0], hrap_coords[1])

  return hrap_idx

def nohrsc_ll2ij(lon,lat,gridlons,gridlats):
  #for a lat/lon grid
  lon = float(lon)
  lat = float(lat)
  lonidx=(np.abs(lon-gridlons)).argmin()
  latidx=(np.abs(lat-gridlats)).argmin()
  return(latidx,lonidx)

def get_stageiv():
  siv_url = "https://water.weather.gov/precip/downloads/"+valid_date_end.strftime('%Y')+"/"+valid_date_end.strftime('%m')+"/"+valid_date_end.strftime('%d')+"/nws_precip_1day_"+valid_date_end.strftime('%Y%m%d')+"_conus.nc"
  data = urlopen(siv_url).read()

  nc = Dataset('data', memory=data)
  #with Dataset(siv_file, 'r') as nc:
  stageIV = nc.variables['observation']
  s4x = nc.variables['x']
  s4y = nc.variables['y']
  return stageIV, s4x, s4y

def get_nohrsc():
  nohrsc_url = "https://www.nohrsc.noaa.gov/snowfall_v2/data/"+valid_date_end.strftime('%Y%m')+"/sfav2_CONUS_24h_"+valid_date_end.strftime('%Y%m%d%H')+".nc"
  data = urlopen(nohrsc_url).read()

  nc = Dataset('data',memory=data)
  snow=np.asarray(nc.variables['Data']) #make lon by lat array (original lat by lon)
  snowlat = np.asarray(nc.variables['lat'])
  snowlon = np.asarray(nc.variables['lon'])
  return snow,snowlon,snowlat

def K_to_F(kelvin):
  fahrenheit = 1.8*(kelvin-273)+32.
  return fahrenheit

def mps_to_kts(mps):
  kts = mps * 1.94384
  return kts

def mm_to_in(millimeters):
  inches = millimeters * 0.0393701
  return inches

def meters_to_in(meters):
  inches = meters*39.3701
  return inches

def find_roots(x,y):
  s = np.abs(np.diff(np.sign(y))).astype(bool)
  return x[:-1][s] + np.diff(x)[s]/(np.abs(y[1:][s]/y[:-1][s])+1)


In [ ]:
#@title Download Obs

synoptic_token = "62e9269f0a164da1b2415ddcf8f4f29e"
statistics_api = "https://api.synopticlabs.org/v2/stations/statistics?"
precipitation_api = "https://api.synopticdata.com/v2/stations/precipitation?"
metadata_api = "https://api.synopticdata.com/v2/stations/metadata?"

print('Getting obs...')
obs ={}

for region in region_list:
  if (valid_end_datetime <= current_datetime):
    print("   > Grabbing obs for: ", region)
    #print("List of CWAs: ", cwa_list(region) )
    json_name = "obs/Obs_"+element+"_"+valid_date_start.strftime('%Y%m%d')+obs_start_hour+"_"+valid_date_end.strftime('%Y%m%d')+obs_end_hour+"_"+region+".json"
    if os.path.exists("obs"):
      pass
    else:
      os.mkdir("obs")
    if element == "mint" or element == "maxt":
      api_token = "&token="+synoptic_token
      station_query = "&cwa="+cwa_list(region)
      vars_query = "&vars=air_temp"
      start_query = "&start="+valid_date_start.strftime('%Y%m%d')+obs_start_hour
      end_query = "&end="+valid_date_end.strftime('%Y%m%d')+obs_end_hour
      stat_type = "&type="+ob_stat
      network_query = network_string
      api_extras = "&units=temp%7Cf&within=1440&status=active"
      obs_url = statistics_api + api_token + station_query + vars_query + start_query + end_query + stat_type + network_query + api_extras
    elif element == "maxwind":
      api_token = "&token="+synoptic_token
      station_query = "&cwa="+cwa_list(region)
      vars_query = "&vars=wind_speed"
      start_query = "&start="+valid_date_start.strftime('%Y%m%d')+obs_start_hour
      end_query = "&end="+valid_date_end.strftime('%Y%m%d')+obs_end_hour
      stat_type = "&type="+ob_stat
      network_query = network_string
      obs_url = statistics_api + api_token + station_query + vars_query + start_query + end_query + stat_type + network_query
    elif element == "qpf":
      if use_stageiv:
        api_token = "&token="+synoptic_token
        station_query = "&cwa="+cwa_list(region)
        api_extras = "&fields=status,latitude,longitude,name,elevation"
        network_query = network_string
        obs_url = metadata_api + api_token + station_query + network_query + api_extras
        stageIV, s4xs, s4ys = get_stageiv()
        s4xs, s4ys = np.meshgrid(s4xs, s4ys)
      else:
        api_token = "&token="+synoptic_token
        station_query = "&cwa="+cwa_list(region)
        api_extras = "&fields=status,latitude,longitude,name,elevation&obtimezone=utc"
        network_query = network_string
        vars_query = "&pmode=totals"
        units_query = "&units=precip|in"
        start_query = "&start="+valid_date_start.strftime('%Y%m%d')+obs_start_hour
        end_query = "&end="+valid_date_end.strftime('%Y%m%d')+obs_end_hour
        obs_url = precipitation_api + api_token + station_query + network_query + vars_query + start_query + end_query + units_query + api_extras
    elif element == "snow":
      if use_nohrsc:
        api_token = "&token="+synoptic_token
        station_query = "&cwa="+cwa_list(region)
        api_extras = "&fields=status,latitude,longitude,name,elevation"
        network_query = network_string
        obs_url = metadata_api + api_token + station_query + network_query + api_extras
        snow,snowlon,snowlat = get_nohrsc()
        snowlons,snowlats = np.meshgrid(snowlon,snowlat)
      else:
        api_token = "&token="+synoptic_token
        station_query = "&cwa="+cwa_list(region)
        api_extras = "&fields=status,latitude,longitude,name,elevation&obtimezone=utc"
        network_query = network_string
        vars_query = "&pmode=totals"
        units_query = "&units=precip|in"
        start_query = "&start="+valid_date_start.strftime('%Y%m%d')+obs_start_hour
        end_query = "&end="+valid_date_end.strftime('%Y%m%d')+obs_end_hour
        obs_url = precipitation_api + api_token + station_query + network_query + vars_query + start_query + end_query + units_query + api_extras
    print("Obs url: " + obs_url)
    if os.path.exists(json_name):
      print ("Skipping download since JSON file already exists")
      pass
    else:
      urlretrieve(obs_url, json_name)

    if os.path.exists(json_name):
        with open(json_name) as json_file:
            obs_json = json.load(json_file)
            print ("Loaded Obs JSON file line 343: " + json_name)
            obs_lats = []
            obs_lons = []
            obs_value = []
            obs_elev = []
            obs_stid = []
            obs_name = []
            for stn in obs_json["STATION"]:
                if stn["STID"] is None:
                  stid = "N0N3"
                else:
                  stid = stn["STID"]
                name = stn["NAME"]
                if stn["ELEVATION"] and stn["ELEVATION"] is not None:
                  elev = stn["ELEVATION"]
                else:
                  elev = -999
                lat = stn["LATITUDE"]
                lon = stn["LONGITUDE"]
                if float(lon) > -50:
                  continue #bug fix to deal with errant synoptic labs obs in the file
                if element == "mint" or element=="maxt":
                  if 'air_temp_set_1' in stn['STATISTICS'] and stn['STATISTICS']['air_temp_set_1']:
                    if ob_stat in stn['STATISTICS']['air_temp_set_1']: # and float(stn["LATITUDE"]) != 0. and float(stn["LONGITUDE"]) != 0.:
                      stat = stn['STATISTICS']['air_temp_set_1'][ob_stat]
                      obs_stid.append(str(stid))
                      obs_name.append(str(name))
                      obs_elev.append(float(elev))
                      obs_lats.append(float(lat))
                      obs_lons.append(float(lon))
                      obs_value.append(float(stat))
                elif element == "maxwind":
                  if 'wind_speed_set_1' in stn['STATISTICS'] and stn['STATISTICS']['wind_speed_set_1']:
                    if ob_stat in stn['STATISTICS']['wind_speed_set_1']: # and float(stn["LATITUDE"]) != 0.:
                      stat = stn['STATISTICS']['wind_speed_set_1'][ob_stat]
                      obs_stid.append(str(stid))
                      obs_name.append(str(name))
                      obs_elev.append(float(elev))
                      obs_lats.append(float(lat))
                      obs_lons.append(float(lon))
                      obs_value.append(mps_to_kts(float(stat)))
                elif (element == "qpf"):
                  if (stn["STATUS"] == "ACTIVE"): # and float(stn["LATITUDE"]) < 50.924 and float(stn["LATITUDE"]) > 23.377 and float(stn["LONGITUDE"]) > -125.650 and float(stn["LONGITUDE"]) < -66.008:
                    obs_stid.append(str(stid))
                    obs_name.append(str(name))
                    obs_elev.append(float(elev))
                    obs_lats.append(float(lat))
                    obs_lons.append(float(lon))
                    if use_stageiv:
                      coords = project_hrap(lon, lat, s4xs, s4ys)
                      siv_value = float(stageIV[coords])
                      if (siv_value >= 0.01):
                        obs_value.append(siv_value)
                      else:
                        obs_value.append(0.0)
                    else:
                      if "precipitation" in stn["OBSERVATIONS"]:
                        if "total" in stn["OBSERVATIONS"]["precipitation"][0]:
                          ptotal = stn["OBSERVATIONS"]["precipitation"][0]["total"]
                          if ptotal >= 0.005:
                            obs_value.append(ptotal)
                          else:
                            obs_value.append(0.0)
                        else:
                          obs_value.append(np.nan)
                      else:
                        obs_value.append(np.nan)
                elif (element == "snow"):
                  if stn["STATUS"] == "ACTIVE": # and float(stn["LATITUDE"]) < 50.924)and float(stn["LATITUDE"]) > 23.377 and float(stn["LONGITUDE"]) > -125.650 and float(stn["LONGITUDE"]) < -66.008:
                    obs_stid.append(str(stid))
                    obs_name.append(str(name))
                    obs_elev.append(float(elev))
                    obs_lats.append(float(lat))
                    obs_lons.append(float(lon))
                    if use_nohrsc:
                      coords = nohrsc_ll2ij(lon,lat,snowlon,snowlat)
                      nohrsc_value = meters_to_in(float(snow[coords]))
                      if nohrsc_value >= 0.005:
                        obs_value.append(nohrsc_value)
                      elif nohrsc_value < 0.0:
                        obs_value.append(np.nan)
                      else:
                        obs_value.append(0.0)
                    else:
                      raise Exception("Still not able to process individual snow obs!")
            csv_name = "obs_"+element+"_"+region+"_"+valid_date_end.strftime('%Y%m%d')+".csv"
            obs[region] = pd.DataFrame()
            obs[region]["stid"] = obs_stid
            obs[region]["name"] = obs_name
            obs[region]["elevation"] = obs_elev
            obs[region]["lat"] = obs_lats
            obs[region]["lon"] = obs_lons
            obs[region]["ob_"+element] = obs_value
            obs[region].dropna(inplace=True)
            obs[region].to_csv(csv_name)
  else:
    print(f'    > Valid Time in the future. Grabbing obs points only for: {region}')
    json_name = "obs/ObsPoints_"+region+"_wcoss.json"
    if os.path.exists(json_name):
      pass
    else:
      if os.path.exists("obs"):
        pass
      else:
        os.mkdir("obs")
      obs_url = "https://api.synopticdata.com/v2/stations/metadata?&token="+synoptic_token+"&cwa="+cwa_list(region)+"&fields=status,latitude,longitude,name,elevation"+network_string
      urlretrieve(obs_url, json_name)
    if os.path.exists(json_name):
      with open(json_name) as json_file:
          obs_json = json.load(json_file)
          print("Loaded Obs JSON file line 197!")
          obs_lats = []
          obs_lons = []
          obs_elev = []
          obs_stid = []
          obs_name = []
          for stn in obs_json["STATION"]:
            # print(stn.encode('utf-8'))
            if stn["STID"] is None:
              stid = "N0N3"
            else:
              stid = stn["STID"]
            #print(f'Processing {region} station {stid}')
            name = stn["NAME"]
            if stn["ELEVATION"] and stn["ELEVATION"] is not None:
              elev = stn["ELEVATION"]
            else:
              elev = -999
            lat = stn["LATITUDE"]
            lon = stn["LONGITUDE"]
            if stn["STATUS"] == "ACTIVE": # and float(stn["LATITUDE"]) != 0. and float(stn["LONGITUDE"]) != 0.:
              obs_stid.append(str(stid))
              obs_name.append(str(name))
              obs_elev.append(float(elev))
              obs_lats.append(float(lat))
              obs_lons.append(float(lon))
          obs[region] = pd.DataFrame()
          obs[region]["stid"] = obs_stid
          obs[region]["name"] = obs_name
          obs[region]["elevation"] = obs_elev
          obs[region]["lat"] = obs_lats
          obs[region]["lon"] = obs_lons
          obs[region]["ob_"+element] = -999
          obs[region].dropna(inplace=True)
          #obs[region].to_csv(csv_name)

#end up with one master frame of all obs
masterobs=pd.concat(obs.values(),ignore_index=True)
masterobs.to_csv("allobs.csv")

In [ ]:
#@title Download Gridded Data

def getthresh(element):
	if element == "gust" or element == "maxgust":
		#kts:m/s
		idthresh = {
			22:11.32,
			34:17.49,
			41:21.09,
			48:24.69,
			56:28.81,
			64:32.92
		}
	elif element == "wind" or element == "maxwind":
		#kts:m/s
		idthresh = {
			11:5.66,
			17:8.75,
			22:11.32,
			34:17.49,
			48:24.69,
			64:32.92
		}
	elif element == "maxt":
		#F:K
		idthresh = {
			0:255,
			28:270,
			32:273,
			80:299,
			90:305,
			100:310,
			110:316,
			120:322
		}
	elif element == "mint":
		#F:K
		idthresh = {
			-40:233,
			-20:244,
			0:255,
			10:260,
			28:270,
			32:273
			#80:299
		}
	elif element == "qpf": #24-H QPF
		#in:in, no conversion necessary
		idthresh = {
			0.01:0.01,
			0.25:0.25,
			0.50:0.50,
			1.00:1.00,
			2.00:2.00,
			3.00:3.00,
			4.00:4.00,
			5.00:5.00,
			6.00:6.00,
			8.00:8.00
		}
	elif element == "qpf06":
		idthresh = {
			0.1:0.1,
			0.25:0.25,
			0.50:0.50,
			0.75:0.75,
			1.00:1.00,
			1.50:1.50,
			2.00:2.00,
			2.50:2.50,
			3.00:3.00
		}
	elif element == "snow": #snow 24
		idthresh = {
			0.1:0.1,
			0.3:0.3,
			0.5:0.5,
			0.7:0.7,
			1.0:1.0,
			1.5:1.5,
			2.0:2.0,
			2.5:2.5,
			3.0:3.0,
			4.0:4.0,
			6.0:6.0,
			8.0:8.0,
			10.0:10.0,
			12.0:12.0,
			18.0:18.0,
			24.0:24.0,
			30.0:30.0,
			36.0:36.0,
			48.0:48.0
		}
	elif element == "snow6":
		idthresh = {
			0.1:0.1,
			0.3:0.3,
			0.5:0.5,
			0.7:0.7,
			1.0:1.0,
			1.5:1.5,
			2.0:2.0,
			2.5:2.5,
			3.0:3.0,
			4.0:4.0,
			5.0:5.0,
			6.0:6.0,
			8.0:8.0,
			10.0:10.0,
			12.0:12.0,
			18.0:18.0,
			24.0:24.0,
			30.0:30.0,
			36.0:36.0,
			48.0:48.0
		}
	elif element in ["ice6","ice"]:
		idthresh = {
			0.01:0.01,
			0.1:0.1,
			0.25:0.25,
			0.50:0.50,
			1.00:1.00
		}
	else:
		raise Exception ("No thresholds in place yet for element " + element + ".  Look at convert.py to add thresholds as needed.")
	return idthresh


def download_subset(remote_url, remote_file, local_filename):
  print("   > Downloading a subset of NBM gribs")
  local_file = "nbm/"+local_filename
  if "qmd" in remote_file:
    if element == "maxt":
      if (int(nbm_qmd_forecasthour_start) % 24 == 0) and (int(nbm_qmd_forecasthour) % 24 ==0):
        search_string = f':TMP:2 m above ground:{str(int(int(nbm_qmd_forecasthour_start)/24))}-{str(int(int(nbm_qmd_forecasthour)/24))} day max fcst:'
      else:
        search_string = f':TMP:2 m above ground:{str(int(nbm_qmd_forecasthour_start))}-{str(int(nbm_qmd_forecasthour))} hour max fcst:'
    elif element == "mint":
      if (int(nbm_qmd_forecasthour_start) % 24 == 0) and (int(nbm_qmd_forecasthour) % 24 ==0):
        search_string = f':TMP:2 m above ground:{str(int(int(nbm_qmd_forecasthour_start)/24))}-{str(int(int(nbm_qmd_forecasthour)/24))} day min fcst:'
      else:
        search_string = f':TMP:2 m above ground:{str(int(nbm_qmd_forecasthour_start))}-{str(int(nbm_qmd_forecasthour))} hour min fcst:'
    elif element == "qpf":
      if (int(nbm_qmd_forecasthour_start) % 24 == 0) and (int(nbm_qmd_forecasthour) % 24 ==0):
        search_string = f':APCP:surface:{str(int(int(nbm_qmd_forecasthour_start)/24))}-{str(int(int(nbm_qmd_forecasthour)/24))} day acc fcst:'
      else:
        search_string = f':APCP:surface:{str(int(nbm_qmd_forecasthour_start))}-{str(int(nbm_qmd_forecasthour))} hour acc fcst:'
    elif element == "maxwind":
      if (int(nbm_qmd_forecasthour_start) % 24 == 0) and (int(nbm_qmd_forecasthour) % 24 == 0):
        search_string = f':WIND:10 m above ground:{str(int(nbm_qmd_forecasthour_start/24))}-{str(int(nbm_qmd_forecasthour/24))} hour max fcst:'
      else:
        search_string = f':WIND:10 m above ground:{str(int(nbm_qmd_forecasthour_start))}-{str(int(nbm_qmd_forecasthour))} hour max fcst:'
  elif "core" in remote_file:
    if element == "maxt":
      search_string = f':TMAX:2 m above ground:{str(int(nbm_core_forecasthour_start))}-{str(int(nbm_core_forecasthour))} hour max fcst:'
    elif element == "mint":
      search_string = f':TMIN:2 m above ground:{str(int(nbm_core_forecasthour_start))}-{str(int(nbm_core_forecasthour))} hour min fcst:'
    elif element == "snow":
      search_string = f':ASNOW:surface:{str(int(nbm_core_forecasthour_start))}-{str(int(nbm_core_forecasthour))} hour acc'
  #print("Search string = ",search_string)
  idx = remote_url+".idx"
  #print("IDX file = " + idx)
  r = requests.get(idx)
  if not r.ok:
    print('     ❌ SORRY! Status Code:', r.status_code, r.reason)
    print(f'      ❌ It does not look like the index file exists: {idx}')

  lines = r.text.split('\n')
  expr = re.compile(search_string)
  expr
  byte_ranges = {}
  for n, line in enumerate(lines, start=1):
    # n is the line number (starting from 1) so that when we call for
    # `lines[n]` it will give us the next line. (Clear as mud??)
    # Use the compiled regular expression to search the line
    #print(">> Searching throgh this line: " + line)
    if expr.search(line):
      # aka, if the line contains the string we are looking for...
      # Get the beginning byte in the line we found
      parts = line.split(':')
      rangestart = int(parts[1])
      # Get the beginning byte in the next line...
      if n+1 < len(lines):
        # ...if there is a next line
        parts = lines[n].split(':')
        rangeend = int(parts[1])
      else:
        # ...if there isn't a next line, then go to the end of the file.
        rangeend = ''

        # Store the byte-range string in our dictionary,
        # and keep the line information too so we can refer back to it.
      byte_ranges[f'{rangestart}-{rangeend}'] = line
      #print(line)
    #else:
      #print(">>>  Could not find search string!")
  #print(">>  Number of items in byteRange:" + str(len(byte_ranges)))
  for i, (byteRange, line) in enumerate(byte_ranges.items()):

    if i == 0:
      # If we are working on the first item, overwrite the existing file.
      curl = f'curl -s --range {byteRange} {remote_url} > {local_file}'
      #print(">>  Adding curl command: " + curl)
    else:
      # If we are working on not the first item, append the existing file.
      curl = f'curl -s --range {byteRange} {remote_url} >> {local_file}'
      #print("Adding curl command: " + curl)
    #print('>>  Parsing line: ' + line)
    try:
      num, byte, date, var, level, forecast, _ = line.split(':')
    except:
      pass
      #print(">>>  Can't get num/byte/etc from this line, so skipping...")

    #print(f'  Downloading GRIB line [{num:>3}]: variable={var}, level={level}, forecast={forecast}')
    #print(f'  Downloading GRIB line: variable={var}, level={level}, forecast={forecast}')
    #print("Running the curl command...")
    os.system(curl)

  if os.path.exists(local_file):
    print(f'      ✅ Success! Searched for [{search_string}] and got [{len(byte_ranges)}] GRIB fields and saved as {local_file}')
    return local_file
  else:
    print(print(f'      ❌ Unsuccessful! Searched for [{search_string}] and did not find anything!'))

########################################################################################################################
# This section downloads and processes the NBM.                                                                        #
########################################################################################################################
if "AR" in region_list or "AJK" in region_list or "ARH" in region_list or "AFC" in region_list:
  rg="ak"
elif "HFO" in region_list:
  rg="hi"
elif "SJU" in region_list:
  rg="pr"
else:
  rg="co"

print('Getting and processing NBM...')
nbm_init_filen = nbm_init.strftime('%Y%m%d') + "_" + nbm_init.strftime('%H')
nbm_init_filen_core = core_init.strftime('%Y%m%d') + "_" + core_init.strftime('%H')
nbm_url_base = "https://noaa-nbm-grib2-pds.s3.amazonaws.com/blend."+nbm_init.strftime('%Y%m%d') \
            +"/"+nbm_init.strftime('%H')+"/"
nbm_url_base_core = "https://noaa-nbm-grib2-pds.s3.amazonaws.com/blend."+core_init.strftime('%Y%m%d') \
            +"/"+core_init.strftime('%H')+"/"
temp_vars = ["maxt","mint"]

prob_dict = {"maxt":"maxt18p", "mint":"mint18p", "qpf":"qpf24p", "snow":"snow24p"}
if element == "snow":
  prob_file=f"blend.t{int(core_init.strftime('%H')):02}z.core.f{int(nbm_core_forecasthour):03}.{rg}.grib2"
  prob_url=nbm_url_base_core+"core/"+prob_file
else:
  prob_file = f'blend.t{int(nbm_init_hour):02}z.qmd.f{int(nbm_qmd_forecasthour):03}.{rg}.grib2'
  prob_url = nbm_url_base+"qmd/"+prob_file
prob_file_subset = f'blend.t{int(nbm_init_hour):02}z.qmd.{nbm_init_filen}{nbm_init_filen}f{int(nbm_qmd_forecasthour):03}.{rg}.{element}_subset.grib2'
print("prob_url=",prob_url)
print("prob_file=",prob_file)
if os.path.exists("nbm"):
  pass
else:
  os.mkdir("nbm")
#Raise exception for missinge percentile data
if rg in ["pr","hi"] and element in ["maxt","mint","maxwind","snow"]:
  raise Exception ("FATAL ERROR: Percentiles for " + element + "do not exist for PR or HI")
if os.path.exists("nbm/"+prob_file_subset):
  print("   > NBM probabilistic already exists")
else:
  #urlretrieve(perc_url, "nbm/"+perc_file)
  print("   > Getting NBM probabilistic")
  download_subset(prob_url, prob_file, prob_file_subset)



In [ ]:
#@title Parse Downloaded Probabilities to Obs (without map)

nbmprob = pygrib.open("nbm/"+prob_file_subset)
print('   > Extracting NBM Probabilistic')

idthresh=getthresh(element)
probfield={}
point_lats=masterobs["lat"]
point_lons=masterobs["lon"]

#initial read of one random field for lat/lon info and identify index/grid values
nbm_fidx=[]
nbmprob.seek(2)
tmpinv=nbmprob.read(2)[0]
nbmlats,nbmlons=tmpinv.latlons()
for i in range(0,len(point_lats)):
  coords = ll_to_index(nbmlons,nbmlats,point_lons[i],point_lats[i])
  nbm_fidx.append(coords)
masterobs["NBM_fidx"] = nbm_fidx

#now read probabilities and add to dataframe
for thresh in idthresh.keys():
  tval=idthresh[thresh]
  if element == "maxt":
    limit=tval
    if limit < 280:
      probinv = nbmprob.select(name="Maximum temperature at 2 metres since previous post-processing",probabilityTypeName="Probability of event below lower limit",lowerLimit=limit)[0]
      prob_name="Prob " + element + " < " + str(thresh)
    else:
      probinv = nbmprob.select(name="Maximum temperature at 2 metres since previous post-processing",probabilityTypeName="Probability of event above upper limit",upperLimit=limit)[0]
      prob_name="Prob " + element + " > " + str(thresh)
    probdata = probinv.values
  elif element == "mint":
    limit=tval
    if limit < 280:
      probinv = nbmprob.select(name="Minimum temperature at 2 metres since previous post-processing",probabilityTypeName="Probability of event below lower limit",lowerLimit=limit)[0]
      prob_name="Prob " + element + " < "+ str(thresh)
    else:
      probinv = nbmprob.select(name="Minimum temperature at 2 metres since previous post-processing",probabilityTypeName="Probability of event above upper limit",upperLimit=limit)[0]
      prob_name="Prob " + element + " > " + str(thresh)
    probdata = probinv.values
  elif element == "qpf":
    limit=tval
    probinv = nbmprob.select(name="Total Precipitation",lengthOfTimeRange=24,probabilityTypeName="Probability of event above upper limit",upperLimit=limit)[0]
    probdata = probinv.values
    prob_name="Prob " + element + " > " + str(thresh)
  elif element == "maxwind":
    limit=tval
    probinv = nbmprob.select(name="10 metre wind speed",lengthOfTimeRange=24,probabilityTypeName="Probability of event above upper limit",upperLimit=limit)[0]
    probdata = probinv.values
    prob_name="Prob " + element + " > " + str(thresh)
  elif element == "snow":
    limit=tval
    probinv = nbmprob.select(name="unknown",lengthOfTimeRange=24,probabilityTypeName="Probability of event above upper limit",upperLimit=limit)[0]
    probdata = probinv.values
    prob_name="Prob " + element + " > " + str(thresh)
  print("    >>  Extracted " + prob_name)
  nbm_coords = masterobs["NBM_fidx"].values
  prob_values = []
  for i in range(0, len(nbm_coords)):
    prob_value = probdata[nbm_coords[i]]
    prob_values.append(prob_value)
  masterobs[prob_name] = prob_values
  probfield[thresh] = probdata
nbmprob.close()
masterobs.to_csv("allobs_withprobs.csv")

In [ ]:
#@title Extract Probabilities and Make Maps

try:
  nbmprob
except:
  nbmprob = pygrib.open("nbm/"+prob_file_subset)
  print('   > Extracting NBM Probabilistic')
  nbmprob.seek(2)
  tmpinv=nbmprob.read(2)[0]
  nbmlats,nbmlons=tmpinv.latlons()

idthresh=getthresh(element)

valdate=valid_end_datetime

matplotlib.rc('axes',facecolor=background_color,edgecolor=text_color)
fig_valid_date=valdate.strftime('%Y%m%d_%HZ')
valid_title=valdate.strftime('%HZ %a %m-%d-%Y')
nbm_init_title=nbm_init.strftime('%HZ %m-%d-%Y')
nbm_init_string=nbm_init.strftime('%Y%m%d_%H') + "Z"

print("Making plot for custom region!")
west = np.nanmin(masterobs["lon"]) - 1.0
south = np.nanmin(masterobs["lat"]) - 1.0
east = np.nanmax(masterobs["lon"]) + 1.0
north = np.nanmax(masterobs["lat"]) + 1.0
width, height = (16,9)
ratioxy = 16./9.
width_ratios = [ratioxy]#, 1]
proj = ccrs.PlateCarree()
print("Plot dimensions have been set up!")
plevels=[1,10,20,30,40,50,60,70,80,90,99]

for thresh in idthresh.keys():
  cust_labels=[]
  print("Looking at threshold: " + str(thresh))
  if element == "mint":
    fieldName="Minimum temperature at 2 metres since previous post-processing"
    if thresh <= 32:
      obsplot=masterobs[masterobs["ob_mint"] < thresh]
      probName="Probability of event below lower limit"
      tdir="< "
    else:
      obsplot=masterobs[masterobs["ob_mint"] > thresh]
      probName="Probability of event above upper limit"
      tdir="> "
  elif element == "maxt":
    fieldName="Maximum temperature at 2 metres since previous post-processing"
    if thresh < 80:
      obsplot=masterobs[masterobs["ob_maxt"] < thresh]
      probName="Probability of event below lower limit"
      tdir="< "
    else:
      obsplot=masterobs[masterobs["ob_maxt"] > thresh]
      probName="Probability of event above upper limit"
      tdir="> "
  elif element == "qpf":
    fieldName="Total Precipitation"
    obsplot=masterobs[masterobs["ob_qpf"] > thresh]
    probName="Probability of event above upper limit"
    tdir="> "
  elif element == "snow":
    fieldName="unknown"
    obsplot=masterobs[masterobs["ob_snow"] > thresh]
    probName="Probability of event above upper limit"
    tdir="> "
  elif element == "maxwind":
    fieldName="10 metre wind speed"
    obsplot=masterobs[masterobs["ob_maxwind"] > thresh]
    probName="Probability of event above upper limit"
    tdir="> "
  #if tdir == "> ":
  #  probfield = nbmprob.select(name=fieldName,probabilityTypeName=probName,upperLimit=thresh)[0]
  #  probdata = probfield.values
  #else:
  #  probfield = nbmprob.select(name=fieldName,probabilityTypeName=probName,lowerLimit=thresh)[0]
  #  probdata = probfield.values
  fig = plt.figure(constrained_layout=True, figsize=(width,height), facecolor=background_color, frameon=True, dpi=150)
  grid = fig.add_gridspec(1,1, hspace=0.1, width_ratios=width_ratios, height_ratios = [1], wspace=0.2)
  ax1 = fig.add_subplot(grid[0,0], projection=ccrs.Mercator())
  ax1.set_anchor('N')
  ax1.set_facecolor(background_color)
  ax1.set_extent([west, east, south, north], crs=proj)
  ax1.add_feature(cfeature.OCEAN, edgecolor='none', facecolor=map_water_color, zorder=-2)
  ax1.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '50m', edgecolor='none', facecolor=map_land_color, zorder=-1))
  #ax1.add_feature(cfeature.LAKES, edgecolor='none', facecolor='#272727', zorder=0)
  ax1.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', edgecolor='none', facecolor=map_water_color, zorder=0))
  ax1.add_feature(cfeature.BORDERS, edgecolor=map_border_color, facecolor='none', linewidth=2, zorder=1)
  #ax1.add_feature(cfeature.NaturalEarthFeature('cultural', 'countries', '50m', edgecolor=map_border_color, facecolor='none', linewidth=2, zorder=2))
  ax1.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_1_states_provinces_lines', '50m', edgecolor=map_border_color, facecolor='none', linewidth=1, zorder=2))
  hits=ax1.scatter(obsplot["lon"].values,obsplot["lat"].values,color="white",s=4,transform=proj,label="Observed Event")
  probs=ax1.contour(nbmlons,nbmlats,probfield[thresh],levels=plevels,cmap=plt.get_cmap('jet'),alpha=0.75,linewidths=1.5,linestyles='solid',transform=proj)
  #plt.clabel(probs,inline=1,fontsize=10)
  for i in range(len(plevels)):
    probs.collections[i].set_label(str(plevels[i]) + "% Probability")
    cust_labels.append(str(plevels[i]) + "% Probability")
  #generate legend
  legtitle="Prob " + element + " " + tdir + str(thresh)
  #for simplicity, we will append scatter plot artist to contour plot artists, then make one legend
  artists,labels=probs.legend_elements()
  artists.append(hits)
  #note that we append to and use plabels, which are our custom labels.
  cust_labels.append("Observed Event")
  legend2 = ax1.legend(artists,cust_labels,loc="best",title=legtitle,fancybox=True)
  #now, just formatting
  for text in legend2.get_texts():
    text.set_color(text_color)
  title=legend2.get_title()
  title.set_color(text_color)
  ax1.add_artist(legend2)
  ##add titles
  fig.text(0.5,1.05,"Obs and Probs: NBM 4.1 " + element + " " + tdir + str(thresh),horizontalalignment='center', verticalalignment='bottom', weight='bold',fontsize=20,color=text_color)
  fig.text(0.5,1.05,"Valid ending at: " + valid_title + " | NBM Init: " + nbm_init_title,horizontalalignment='center',verticalalignment='top', fontsize=16,color=text_color)
  ##save figure
  figname="obs_and_probs_v4.1_"+region+"_"+element+"_"+str(thresh)+"_"+nbm_init_string+"_"+fig_valid_date+".png"
  plt.savefig(figname,bbox_inches='tight',pad_inches=0.2,dpi='figure')
  print("   >Done! Saved plot as " + figname)

#hitframe=obframe[obframe["Hit_1.0"] == "HIT"]
#hitscat=ax1.scatter(hitframe["lon"].values,hitframe["lat"].values,color="limegreen",transform=proj)